In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import warnings
warnings.filterwarnings('ignore')

### Usaremos la información de los resultados de la liga española para ver algunos ejemplos de predicción 

In [2]:
laliga2122 = pd.read_excel('datos/LaLiga21 22.xlsx', )
laliga2021 = pd.read_excel('datos/LaLiga20 21.xlsx', )
laliga1920 = pd.read_excel('datos/LaLiga19 20.xlsx', )
laliga1819 = pd.read_excel('datos/LaLiga18 19.xlsx', )
laliga1718 = pd.read_excel('datos/LaLiga17 18.xlsx', )
laliga1617 = pd.read_excel('datos/LaLiga16 17.xlsx', )

In [3]:
laliga = laliga2122.dropna(how='all').append(laliga2021.dropna(how='all')
).append(laliga1920.dropna(how='all')).append(laliga1819.dropna(how='all')
).append(laliga1718.dropna(how='all')).append(laliga1617.dropna(how='all')).loc[:,
['Wk','Date','Local','Score','Visitante', 'Attendance']].reset_index(drop = True)
laliga['Attendance']= laliga['Attendance']*1000

In [4]:
def golesvisita(marcador):
    goles = marcador.split('–')
    return int(goles[1])
def goleslocal(marcador):
    goles = marcador.split('–')
    return int(goles[0])

def torneo(año,mes):
    if ((año==2016) & (mes<=12)|(año==2017) & (mes<8)):
        return '16/17'
    elif ((año==2017) & (mes<=12)|(año==2018) & (mes<8)):
        return '17/18'
    elif ((año==2018) & (mes<=12)|(año==2019) & (mes<8)):
        return '18/19'
    elif ((año==2019) & (mes<=12)|(año==2020) & (mes<9)):
        return '19/20'
    elif ((año==2020) & (mes<=12)|(año==2021) & (mes<8)):
        return '20/21'
    elif ((año==2021) & (mes<=12)|(año==2022) & (mes<8)):
        return '21/22'

def resultado(string):
    goles = string.split('–')
    if int(goles[0])>int(goles[1]):
        return 'Local'
    if int(goles[0])<int(goles[1]):
        return 'Visitante'
    else: 
        return 'Empate'
    
def resultado_equipo(string, local, visita):
    goles = string.split('–')
    if int(goles[0])>int(goles[1]):
        return local
    if int(goles[0])<int(goles[1]):
        return visita
    else: 
        return 'Empate'
    

laliga['Goles Local'] = laliga['Score'].apply(goleslocal)
laliga['Goles Visitante'] = laliga['Score'].apply(golesvisita)
laliga['Total goles partido']= laliga['Goles Local']+laliga['Goles Visitante']
laliga['Año'] = laliga['Date'].dt.year
laliga['Mes'] = laliga['Date'].dt.month
laliga['Temporada'] = laliga.apply(lambda x: torneo(x['Año'],x['Mes']),axis = 1)
laliga['Resultado'] = laliga['Score'].apply(resultado)
laliga['Ganador'] = laliga.apply(lambda x: resultado_equipo(x['Score'],x['Local'],x['Visitante']),axis=1)

In [5]:
laliga

,Wk,Date,Local,Score,Visitante,Attendance,Goles Local,Goles Visitante,Total goles partido,Año,Mes,Temporada,Resultado,Ganador
0,1.0,2021-08-13,Valencia,1–0,Getafe,9868.0,1,0,1,2021,8,21/22,Local,Valencia
1,1.0,2021-08-14,Cádiz,1–1,Levante,6941.0,1,1,2,2021,8,21/22,Empate,Empate
2,1.0,2021-08-14,Mallorca,1–1,Betis,6279.0,1,1,2,2021,8,21/22,Empate,Empate
3,1.0,2021-08-14,Osasuna,0–0,Espanyol,6715.0,0,0,0,2021,8,21/22,Empate,Empate
4,1.0,2021-08-14,Alavés,1–4,Real Madrid,3968.0,1,4,5,2021,8,21/22,Visitante,Real Madrid
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2275,38.0,2017-05-21,Valencia,1–3,Villarreal,33587.0,1,3,4,2017,5,16/17,Visitante,Villarreal
2276,38.0,2017-05-21,Atlético Madrid,3–1,Athletic Club,52314.0,3,1,4,2017,5,16/17,Local,Atlético Madrid
2277,38.0,2017-05-21,Celta Vigo,2–2,Real Sociedad,14188.0,2,2,4,2017,5,16/17,Empate,Empate
2278,38.0,2017-05-21,Barcelona,4–2,Eibar,74932.0,4,2,6,2017,5,16/17,Local,Barcelona


### Comencemos mirando un ejemplo para los partidos del Real Madrid con el Barcelona

In [6]:
rm_bar = laliga[((laliga['Local']=='Real Madrid')&(laliga['Visitante']=='Barcelona')|
        ((laliga['Local']=='Barcelona')&(laliga['Visitante']=='Real Madrid')))].reset_index(drop=True)

In [7]:
rm_bar

,Wk,Date,Local,Score,Visitante,Attendance,Goles Local,Goles Visitante,Total goles partido,Año,Mes,Temporada,Resultado,Ganador
0,10.0,2021-10-24,Barcelona,1–2,Real Madrid,86422.0,1,2,3,2021,10,21/22,Visitante,Real Madrid
1,29.0,2022-03-20,Real Madrid,0–4,Barcelona,60017.0,0,4,4,2022,3,21/22,Visitante,Barcelona
2,7.0,2020-10-24,Barcelona,1–3,Real Madrid,NaN,1,3,4,2020,10,20/21,Visitante,Real Madrid
3,30.0,2021-04-10,Real Madrid,2–1,Barcelona,NaN,2,1,3,2021,4,20/21,Local,Real Madrid
4,10.0,2019-12-18,Barcelona,0–0,Real Madrid,93426.0,0,0,0,2019,12,19/20,Empate,Empate
5,26.0,2020-03-01,Real Madrid,2–0,Barcelona,78357.0,2,0,2,2020,3,19/20,Local,Real Madrid
6,10.0,2018-10-28,Barcelona,5–1,Real Madrid,93265.0,5,1,6,2018,10,18/19,Local,Barcelona
7,26.0,2019-03-02,Real Madrid,0–1,Barcelona,78921.0,0,1,1,2019,3,18/19,Visitante,Barcelona
8,17.0,2017-12-23,Real Madrid,0–3,Barcelona,80264.0,0,3,3,2017,12,17/18,Visitante,Barcelona
9,36.0,2018-05-06,Barcelona,2–2,Real Madrid,97939.0,2,2,4,2018,5,17/18,Empate,Empate


In [8]:
rm_bar[['Temporada','Date','Local','Score','Visitante','Año']]

,Temporada,Date,Local,Score,Visitante,Año
0,21/22,2021-10-24,Barcelona,1–2,Real Madrid,2021
1,21/22,2022-03-20,Real Madrid,0–4,Barcelona,2022
2,20/21,2020-10-24,Barcelona,1–3,Real Madrid,2020
3,20/21,2021-04-10,Real Madrid,2–1,Barcelona,2021
4,19/20,2019-12-18,Barcelona,0–0,Real Madrid,2019
5,19/20,2020-03-01,Real Madrid,2–0,Barcelona,2020
6,18/19,2018-10-28,Barcelona,5–1,Real Madrid,2018
7,18/19,2019-03-02,Real Madrid,0–1,Barcelona,2019
8,17/18,2017-12-23,Real Madrid,0–3,Barcelona,2017
9,17/18,2018-05-06,Barcelona,2–2,Real Madrid,2018


### Usaremos la información de todos lo torneos para entender patrones de comportamiento de los resultados históricos entre estos dos equipos


In [9]:
rm_bar[rm_bar['Año']!=2022]

,Wk,Date,Local,Score,Visitante,Attendance,Goles Local,Goles Visitante,Total goles partido,Año,Mes,Temporada,Resultado,Ganador
0,10.0,2021-10-24,Barcelona,1–2,Real Madrid,86422.0,1,2,3,2021,10,21/22,Visitante,Real Madrid
2,7.0,2020-10-24,Barcelona,1–3,Real Madrid,NaN,1,3,4,2020,10,20/21,Visitante,Real Madrid
3,30.0,2021-04-10,Real Madrid,2–1,Barcelona,NaN,2,1,3,2021,4,20/21,Local,Real Madrid
4,10.0,2019-12-18,Barcelona,0–0,Real Madrid,93426.0,0,0,0,2019,12,19/20,Empate,Empate
5,26.0,2020-03-01,Real Madrid,2–0,Barcelona,78357.0,2,0,2,2020,3,19/20,Local,Real Madrid
6,10.0,2018-10-28,Barcelona,5–1,Real Madrid,93265.0,5,1,6,2018,10,18/19,Local,Barcelona
7,26.0,2019-03-02,Real Madrid,0–1,Barcelona,78921.0,0,1,1,2019,3,18/19,Visitante,Barcelona
8,17.0,2017-12-23,Real Madrid,0–3,Barcelona,80264.0,0,3,3,2017,12,17/18,Visitante,Barcelona
9,36.0,2018-05-06,Barcelona,2–2,Real Madrid,97939.0,2,2,4,2018,5,17/18,Empate,Empate
10,14.0,2016-12-03,Barcelona,1–1,Real Madrid,98485.0,1,1,2,2016,12,16/17,Empate,Empate


### Usaremos la media de goles históricos de cada equipo como predictor de los goles del próximo partido

In [10]:
test1 = rm_bar.groupby('Local')['Goles Local'].mean().reset_index()
test1 = test1.rename(columns={"Local": "Equipo", 'Goles Local': 'Goles promedio de local'})

test2 = rm_bar.groupby('Visitante')['Goles Visitante'].mean().reset_index()
test2 = test2.rename(columns = {'Goles Visitante': 'Goles promedio de visitante'})

result = pd.concat([test1, test2], axis=1)
dt = result.drop(['Visitante'], axis=1)
dt['Goles promedio'] = np.round((dt['Goles promedio de local']+dt['Goles promedio de visitante'])/2)
dt['Goles promedio de local'] = np.round(dt['Goles promedio de local'])
dt['Goles promedio de visitante'] = np.round(dt['Goles promedio de visitante'])
dt

,Equipo,Goles promedio de local,Goles promedio de visitante,Goles promedio
0,Barcelona,2.0,2.0,2.0
1,Real Madrid,1.0,2.0,1.0


### Con esta función crearemos las columnas con la predicción

In [11]:
def nuevo_resultado(rm_bar,goles):
    goles_rm = goles.loc[1,'Goles promedio']
    goles_bar = goles.loc[0,'Goles promedio']

    for i in range(len(rm_bar)):
        if rm_bar.iloc[i]['Local']=='Real Madrid':
            rm_bar.loc[i,'Resultado 2']= str(int(goles_rm))+":"+str(int(goles_bar))
            rm_bar.loc[i,'Goles Local 2']= int(goles_rm)
            rm_bar.loc[i,'Goles Visitante 2'] = int(goles_bar)
            rm_bar.loc[i,'Total goles partido 2'] = rm_bar.loc[i,'Goles Local 2']+ rm_bar.loc[i,'Goles Visitante 2']
        else:
            rm_bar.loc[i,'Resultado 2']= str(int(goles_bar))+":"+str(int(goles_rm))
            rm_bar.loc[i,'Goles Local 2']= int(goles_bar)
            rm_bar.loc[i,'Goles Visitante 2'] = int(goles_rm)
            rm_bar.loc[i,'Total goles partido 2'] = rm_bar.loc[i,'Goles Local 2']+ rm_bar.loc[i,'Goles Visitante 2']
    return rm_bar            

In [12]:
mostrar = nuevo_resultado(rm_bar,dt)
mostrar.sort_values('Date',ascending=False)

,Wk,Date,Local,Score,Visitante,Attendance,Goles Local,Goles Visitante,Total goles partido,Año,Mes,Temporada,Resultado,Ganador,Resultado 2,Goles Local 2,Goles Visitante 2,Total goles partido 2
1,29.0,2022-03-20,Real Madrid,0–4,Barcelona,60017.0,0,4,4,2022,3,21/22,Visitante,Barcelona,1:2,1.0,2.0,3.0
0,10.0,2021-10-24,Barcelona,1–2,Real Madrid,86422.0,1,2,3,2021,10,21/22,Visitante,Real Madrid,2:1,2.0,1.0,3.0
3,30.0,2021-04-10,Real Madrid,2–1,Barcelona,NaN,2,1,3,2021,4,20/21,Local,Real Madrid,1:2,1.0,2.0,3.0
2,7.0,2020-10-24,Barcelona,1–3,Real Madrid,NaN,1,3,4,2020,10,20/21,Visitante,Real Madrid,2:1,2.0,1.0,3.0
5,26.0,2020-03-01,Real Madrid,2–0,Barcelona,78357.0,2,0,2,2020,3,19/20,Local,Real Madrid,1:2,1.0,2.0,3.0
4,10.0,2019-12-18,Barcelona,0–0,Real Madrid,93426.0,0,0,0,2019,12,19/20,Empate,Empate,2:1,2.0,1.0,3.0
7,26.0,2019-03-02,Real Madrid,0–1,Barcelona,78921.0,0,1,1,2019,3,18/19,Visitante,Barcelona,1:2,1.0,2.0,3.0
6,10.0,2018-10-28,Barcelona,5–1,Real Madrid,93265.0,5,1,6,2018,10,18/19,Local,Barcelona,2:1,2.0,1.0,3.0
9,36.0,2018-05-06,Barcelona,2–2,Real Madrid,97939.0,2,2,4,2018,5,17/18,Empate,Empate,2:1,2.0,1.0,3.0
8,17.0,2017-12-23,Real Madrid,0–3,Barcelona,80264.0,0,3,3,2017,12,17/18,Visitante,Barcelona,1:2,1.0,2.0,3.0


### Vemos la "calidad" de nuestra predicción calculando el error absoluto para cada predicción realizada


In [13]:
mostrar['Error absoluto'] = np.abs(mostrar['Total goles partido']-mostrar['Total goles partido 2'])
mostrar.sort_values('Date',ascending=False)

,Wk,Date,Local,Score,Visitante,Attendance,Goles Local,Goles Visitante,Total goles partido,Año,Mes,Temporada,Resultado,Ganador,Resultado 2,Goles Local 2,Goles Visitante 2,Total goles partido 2,Error absoluto
1,29.0,2022-03-20,Real Madrid,0–4,Barcelona,60017.0,0,4,4,2022,3,21/22,Visitante,Barcelona,1:2,1.0,2.0,3.0,1.0
0,10.0,2021-10-24,Barcelona,1–2,Real Madrid,86422.0,1,2,3,2021,10,21/22,Visitante,Real Madrid,2:1,2.0,1.0,3.0,0.0
3,30.0,2021-04-10,Real Madrid,2–1,Barcelona,NaN,2,1,3,2021,4,20/21,Local,Real Madrid,1:2,1.0,2.0,3.0,0.0
2,7.0,2020-10-24,Barcelona,1–3,Real Madrid,NaN,1,3,4,2020,10,20/21,Visitante,Real Madrid,2:1,2.0,1.0,3.0,1.0
5,26.0,2020-03-01,Real Madrid,2–0,Barcelona,78357.0,2,0,2,2020,3,19/20,Local,Real Madrid,1:2,1.0,2.0,3.0,1.0
4,10.0,2019-12-18,Barcelona,0–0,Real Madrid,93426.0,0,0,0,2019,12,19/20,Empate,Empate,2:1,2.0,1.0,3.0,3.0
7,26.0,2019-03-02,Real Madrid,0–1,Barcelona,78921.0,0,1,1,2019,3,18/19,Visitante,Barcelona,1:2,1.0,2.0,3.0,2.0
6,10.0,2018-10-28,Barcelona,5–1,Real Madrid,93265.0,5,1,6,2018,10,18/19,Local,Barcelona,2:1,2.0,1.0,3.0,3.0
9,36.0,2018-05-06,Barcelona,2–2,Real Madrid,97939.0,2,2,4,2018,5,17/18,Empate,Empate,2:1,2.0,1.0,3.0,1.0
8,17.0,2017-12-23,Real Madrid,0–3,Barcelona,80264.0,0,3,3,2017,12,17/18,Visitante,Barcelona,1:2,1.0,2.0,3.0,0.0


In [14]:
round(mostrar['Error absoluto'].mean(),2)

1.25

### Replicaremos el método, considerando solamente los goles totales, sin condición de local o visita, para todos los partidos de la temporada 21/22 

In [15]:
laligasin2122 = laliga[laliga['Temporada']!='21/22']
laligasin2122.sort_values('Temporada',ascending=False)

,Wk,Date,Local,Score,Visitante,Attendance,Goles Local,Goles Visitante,Total goles partido,Año,Mes,Temporada,Resultado,Ganador
380,1.0,2020-09-12,Eibar,0–0,Celta Vigo,NaN,0,0,0,2020,9,20/21,Empate,Empate
631,26.0,2021-03-06,Cádiz,1–0,Eibar,NaN,1,0,1,2021,3,20/21,Local,Cádiz
640,27.0,2021-03-13,Alavés,1–1,Cádiz,NaN,1,1,2,2021,3,20/21,Empate,Empate
639,27.0,2021-03-12,Levante,1–0,Valencia,NaN,1,0,1,2021,3,20/21,Local,Levante
638,18.0,2021-03-10,Atlético Madrid,2–1,Athletic Club,NaN,2,1,3,2021,3,20/21,Local,Atlético Madrid
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024,13.0,2016-11-26,Sevilla,2–1,Valencia,31901.0,2,1,3,2016,11,16/17,Local,Sevilla
2023,13.0,2016-11-26,Espanyol,3–0,Leganés,17137.0,3,0,3,2016,11,16/17,Local,Espanyol
2022,13.0,2016-11-26,Real Madrid,2–1,Sporting Gijón,67118.0,2,1,3,2016,11,16/17,Local,Real Madrid
2021,13.0,2016-11-26,Málaga,4–3,La Coruña,12679.0,4,3,7,2016,11,16/17,Local,Málaga


In [16]:
test1_l = laliga.groupby('Local')['Goles Local'].mean().reset_index()
test1_l = test1_l.rename(columns={"Local": "Equipo", 'Goles Local': 'Goles promedio de local'})

test2_l = laligasin2122.groupby('Visitante')['Goles Visitante'].mean().reset_index()
test2_l = test2_l.rename(columns = {'Goles Visitante': 'Goles promedio de visitante'})

result_l = pd.concat([test1_l, test2_l], axis=1)
dt_l = result_l.drop(['Visitante'], axis=1)
dt_l['Goles promedio'] = np.round((dt_l['Goles promedio de local'] + dt_l['Goles promedio de visitante'])/2)
dt_l['Goles promedio de local'] = np.round(dt_l['Goles promedio de local'])
dt_l['Goles promedio de visitante'] = np.round(dt_l['Goles promedio de visitante'])
dt_l

,Equipo,Goles promedio de local,Goles promedio de visitante,Goles promedio
0,Alavés,1.0,1.0,1.0
1,Athletic Club,1.0,1.0,1.0
2,Atlético Madrid,2.0,1.0,2.0
3,Barcelona,3.0,2.0,2.0
4,Betis,2.0,1.0,1.0
5,Celta Vigo,2.0,1.0,1.0
6,Cádiz,1.0,1.0,1.0
7,Eibar,1.0,1.0,1.0
8,Elche,1.0,1.0,1.0
9,Espanyol,1.0,1.0,1.0


In [17]:
def nuevo_resultado_laliga(goles,laliga):
    laliga2 = laliga.copy()
    for i in range(len(laliga2)):
        local = laliga2.iloc[i]['Local']
        visitante = laliga2.iloc[i]['Visitante']
        index_loc = goles[goles['Equipo']==local].index.values.astype(int)[0]
        index_vis = goles[goles['Equipo']==visitante].index.values.astype(int)[0]
        goles_loc = goles.loc[index_loc,'Goles promedio']
        goles_vis = goles.loc[index_vis,'Goles promedio']
        if laliga2.iloc[i]['Local']== local:
            laliga2.loc[i,'Resultado P1']= str(int(goles_loc))+":"+str(int(goles_vis))
            laliga2.loc[i,'Goles Local P1']= int(goles_loc)
            laliga2.loc[i,'Goles Visitante P1'] = int(goles_vis)
            laliga2.loc[i,'Total goles partido P1'] = laliga2.loc[i,'Goles Local P1'] + laliga2.loc[i,'Goles Visitante P1']
    return laliga2

In [18]:
laliga2 = nuevo_resultado_laliga(dt_l,laliga)
laliga2.sort_values('Temporada',ascending=False)

,Wk,Date,Local,Score,Visitante,Attendance,Goles Local,Goles Visitante,Total goles partido,Año,Mes,Temporada,Resultado,Ganador,Resultado P1,Goles Local P1,Goles Visitante P1,Total goles partido P1
0,1.0,2021-08-13,Valencia,1–0,Getafe,9868.0,1,0,1,2021,8,21/22,Local,Valencia,1:1,1.0,1.0,2.0
251,26.0,2022-02-26,Rayo Vallecano,0–1,Real Madrid,9952.0,0,1,1,2022,2,21/22,Visitante,Real Madrid,1:2,1.0,2.0,3.0
260,27.0,2022-03-05,Osasuna,1–0,Villarreal,16659.0,1,0,1,2022,3,21/22,Local,Osasuna,1:2,1.0,2.0,3.0
259,27.0,2022-03-04,Alavés,0–0,Sevilla,13121.0,0,0,0,2022,3,21/22,Empate,Empate,1:2,1.0,2.0,3.0
258,21.0,2022-03-02,Mallorca,0–2,Real Sociedad,11550.0,0,2,2,2022,3,21/22,Visitante,Real Sociedad,1:1,1.0,1.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024,13.0,2016-11-26,Sevilla,2–1,Valencia,31901.0,2,1,3,2016,11,16/17,Local,Sevilla,2:1,2.0,1.0,3.0
2023,13.0,2016-11-26,Espanyol,3–0,Leganés,17137.0,3,0,3,2016,11,16/17,Local,Espanyol,1:1,1.0,1.0,2.0
2022,13.0,2016-11-26,Real Madrid,2–1,Sporting Gijón,67118.0,2,1,3,2016,11,16/17,Local,Real Madrid,2:1,2.0,1.0,3.0
2021,13.0,2016-11-26,Málaga,4–3,La Coruña,12679.0,4,3,7,2016,11,16/17,Local,Málaga,1:1,1.0,1.0,2.0


### y calculamos el error absoluto

In [19]:
CalcularError2122 = laliga2[laliga2['Temporada']=='21/22']
CalcularError2122

,Wk,Date,Local,Score,Visitante,Attendance,Goles Local,Goles Visitante,Total goles partido,Año,Mes,Temporada,Resultado,Ganador,Resultado P1,Goles Local P1,Goles Visitante P1,Total goles partido P1
0,1.0,2021-08-13,Valencia,1–0,Getafe,9868.0,1,0,1,2021,8,21/22,Local,Valencia,1:1,1.0,1.0,2.0
1,1.0,2021-08-14,Cádiz,1–1,Levante,6941.0,1,1,2,2021,8,21/22,Empate,Empate,1:1,1.0,1.0,2.0
2,1.0,2021-08-14,Mallorca,1–1,Betis,6279.0,1,1,2,2021,8,21/22,Empate,Empate,1:1,1.0,1.0,2.0
3,1.0,2021-08-14,Osasuna,0–0,Espanyol,6715.0,0,0,0,2021,8,21/22,Empate,Empate,1:1,1.0,1.0,2.0
4,1.0,2021-08-14,Alavés,1–4,Real Madrid,3968.0,1,4,5,2021,8,21/22,Visitante,Real Madrid,1:2,1.0,2.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,38.0,2022-05-22,Alavés,0–1,Cádiz,11138.0,0,1,1,2022,5,21/22,Visitante,Cádiz,1:1,1.0,1.0,2.0
376,38.0,2022-05-22,Granada,0–0,Espanyol,17951.0,0,0,0,2022,5,21/22,Empate,Empate,1:1,1.0,1.0,2.0
377,38.0,2022-05-22,Real Sociedad,1–2,Atlético Madrid,23586.0,1,2,3,2022,5,21/22,Visitante,Atlético Madrid,1:2,1.0,2.0,3.0
378,38.0,2022-05-22,Barcelona,0–2,Villarreal,54850.0,0,2,2,2022,5,21/22,Visitante,Villarreal,2:2,2.0,2.0,4.0


In [20]:
CalcularError2122['Error absoluto'] = np.abs(CalcularError2122['Total goles partido']-CalcularError2122['Total goles partido P1'])

In [21]:
CalcularError2122

,Wk,Date,Local,Score,Visitante,Attendance,Goles Local,Goles Visitante,Total goles partido,Año,Mes,Temporada,Resultado,Ganador,Resultado P1,Goles Local P1,Goles Visitante P1,Total goles partido P1,Error absoluto
0,1.0,2021-08-13,Valencia,1–0,Getafe,9868.0,1,0,1,2021,8,21/22,Local,Valencia,1:1,1.0,1.0,2.0,1.0
1,1.0,2021-08-14,Cádiz,1–1,Levante,6941.0,1,1,2,2021,8,21/22,Empate,Empate,1:1,1.0,1.0,2.0,0.0
2,1.0,2021-08-14,Mallorca,1–1,Betis,6279.0,1,1,2,2021,8,21/22,Empate,Empate,1:1,1.0,1.0,2.0,0.0
3,1.0,2021-08-14,Osasuna,0–0,Espanyol,6715.0,0,0,0,2021,8,21/22,Empate,Empate,1:1,1.0,1.0,2.0,2.0
4,1.0,2021-08-14,Alavés,1–4,Real Madrid,3968.0,1,4,5,2021,8,21/22,Visitante,Real Madrid,1:2,1.0,2.0,3.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,38.0,2022-05-22,Alavés,0–1,Cádiz,11138.0,0,1,1,2022,5,21/22,Visitante,Cádiz,1:1,1.0,1.0,2.0,1.0
376,38.0,2022-05-22,Granada,0–0,Espanyol,17951.0,0,0,0,2022,5,21/22,Empate,Empate,1:1,1.0,1.0,2.0,2.0
377,38.0,2022-05-22,Real Sociedad,1–2,Atlético Madrid,23586.0,1,2,3,2022,5,21/22,Visitante,Atlético Madrid,1:2,1.0,2.0,3.0,0.0
378,38.0,2022-05-22,Barcelona,0–2,Villarreal,54850.0,0,2,2,2022,5,21/22,Visitante,Villarreal,2:2,2.0,2.0,4.0,2.0


In [22]:
round(CalcularError2122['Error absoluto'].mean(),2)

1.37

### Ahora utilizamos los goles de local y de visita, para ver si nuestra predicción es mejor que la anterior

In [23]:
def nuevo_resultado_laliga2(goles,laliga):
    laliga3 = laliga.copy()
    for i in range(len(laliga2)):
        local = laliga2.iloc[i]['Local']
        visitante = laliga2.iloc[i]['Visitante']
        index_loc = goles[goles['Equipo']==local].index.values.astype(int)[0]
        index_vis = goles[goles['Equipo']==visitante].index.values.astype(int)[0]
        goles_loc = goles.loc[index_loc,'Goles promedio de local']
        goles_vis = goles.loc[index_vis,'Goles promedio de visitante']
        if laliga2.iloc[i]['Local']== local:
            laliga2.loc[i,'Resultado P2']= str(int(goles_loc))+":"+str(int(goles_vis))
            laliga2.loc[i,'Goles Local P2']= int(goles_loc)
            laliga2.loc[i,'Goles Visitante P2'] = int(goles_vis)
            laliga2.loc[i,'Total goles partido P2'] = laliga2.loc[i,'Goles Local P2']+ laliga2.loc[i,'Goles Visitante P2']
    return laliga2

In [24]:
laliga3 = nuevo_resultado_laliga2(dt_l,laliga)

In [25]:
laliga3

,Wk,Date,Local,Score,Visitante,Attendance,Goles Local,Goles Visitante,Total goles partido,Año,...,Resultado,Ganador,Resultado P1,Goles Local P1,Goles Visitante P1,Total goles partido P1,Resultado P2,Goles Local P2,Goles Visitante P2,Total goles partido P2
0,1.0,2021-08-13,Valencia,1–0,Getafe,9868.0,1,0,1,2021,...,Local,Valencia,1:1,1.0,1.0,2.0,2:1,2.0,1.0,3.0
1,1.0,2021-08-14,Cádiz,1–1,Levante,6941.0,1,1,2,2021,...,Empate,Empate,1:1,1.0,1.0,2.0,1:1,1.0,1.0,2.0
2,1.0,2021-08-14,Mallorca,1–1,Betis,6279.0,1,1,2,2021,...,Empate,Empate,1:1,1.0,1.0,2.0,1:1,1.0,1.0,2.0
3,1.0,2021-08-14,Osasuna,0–0,Espanyol,6715.0,0,0,0,2021,...,Empate,Empate,1:1,1.0,1.0,2.0,1:1,1.0,1.0,2.0
4,1.0,2021-08-14,Alavés,1–4,Real Madrid,3968.0,1,4,5,2021,...,Visitante,Real Madrid,1:2,1.0,2.0,3.0,1:2,1.0,2.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2275,38.0,2017-05-21,Valencia,1–3,Villarreal,33587.0,1,3,4,2017,...,Visitante,Villarreal,1:2,1.0,2.0,3.0,2:1,2.0,1.0,3.0
2276,38.0,2017-05-21,Atlético Madrid,3–1,Athletic Club,52314.0,3,1,4,2017,...,Local,Atlético Madrid,2:1,2.0,1.0,3.0,2:1,2.0,1.0,3.0
2277,38.0,2017-05-21,Celta Vigo,2–2,Real Sociedad,14188.0,2,2,4,2017,...,Empate,Empate,1:1,1.0,1.0,2.0,2:1,2.0,1.0,3.0
2278,38.0,2017-05-21,Barcelona,4–2,Eibar,74932.0,4,2,6,2017,...,Local,Barcelona,2:1,2.0,1.0,3.0,3:1,3.0,1.0,4.0


In [26]:
CalcularError2122_V2 = laliga3[laliga3['Temporada']=='21/22']
CalcularError2122_V2

,Wk,Date,Local,Score,Visitante,Attendance,Goles Local,Goles Visitante,Total goles partido,Año,...,Resultado,Ganador,Resultado P1,Goles Local P1,Goles Visitante P1,Total goles partido P1,Resultado P2,Goles Local P2,Goles Visitante P2,Total goles partido P2
0,1.0,2021-08-13,Valencia,1–0,Getafe,9868.0,1,0,1,2021,...,Local,Valencia,1:1,1.0,1.0,2.0,2:1,2.0,1.0,3.0
1,1.0,2021-08-14,Cádiz,1–1,Levante,6941.0,1,1,2,2021,...,Empate,Empate,1:1,1.0,1.0,2.0,1:1,1.0,1.0,2.0
2,1.0,2021-08-14,Mallorca,1–1,Betis,6279.0,1,1,2,2021,...,Empate,Empate,1:1,1.0,1.0,2.0,1:1,1.0,1.0,2.0
3,1.0,2021-08-14,Osasuna,0–0,Espanyol,6715.0,0,0,0,2021,...,Empate,Empate,1:1,1.0,1.0,2.0,1:1,1.0,1.0,2.0
4,1.0,2021-08-14,Alavés,1–4,Real Madrid,3968.0,1,4,5,2021,...,Visitante,Real Madrid,1:2,1.0,2.0,3.0,1:2,1.0,2.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,38.0,2022-05-22,Alavés,0–1,Cádiz,11138.0,0,1,1,2022,...,Visitante,Cádiz,1:1,1.0,1.0,2.0,1:1,1.0,1.0,2.0
376,38.0,2022-05-22,Granada,0–0,Espanyol,17951.0,0,0,0,2022,...,Empate,Empate,1:1,1.0,1.0,2.0,1:1,1.0,1.0,2.0
377,38.0,2022-05-22,Real Sociedad,1–2,Atlético Madrid,23586.0,1,2,3,2022,...,Visitante,Atlético Madrid,1:2,1.0,2.0,3.0,2:1,2.0,1.0,3.0
378,38.0,2022-05-22,Barcelona,0–2,Villarreal,54850.0,0,2,2,2022,...,Visitante,Villarreal,2:2,2.0,2.0,4.0,3:1,3.0,1.0,4.0


In [27]:
CalcularError2122_V2['Error absoluto'] = np.abs(CalcularError2122_V2['Total goles partido']-CalcularError2122_V2['Total goles partido P2'])

In [28]:
CalcularError2122_V2

,Wk,Date,Local,Score,Visitante,Attendance,Goles Local,Goles Visitante,Total goles partido,Año,...,Ganador,Resultado P1,Goles Local P1,Goles Visitante P1,Total goles partido P1,Resultado P2,Goles Local P2,Goles Visitante P2,Total goles partido P2,Error absoluto
0,1.0,2021-08-13,Valencia,1–0,Getafe,9868.0,1,0,1,2021,...,Valencia,1:1,1.0,1.0,2.0,2:1,2.0,1.0,3.0,2.0
1,1.0,2021-08-14,Cádiz,1–1,Levante,6941.0,1,1,2,2021,...,Empate,1:1,1.0,1.0,2.0,1:1,1.0,1.0,2.0,0.0
2,1.0,2021-08-14,Mallorca,1–1,Betis,6279.0,1,1,2,2021,...,Empate,1:1,1.0,1.0,2.0,1:1,1.0,1.0,2.0,0.0
3,1.0,2021-08-14,Osasuna,0–0,Espanyol,6715.0,0,0,0,2021,...,Empate,1:1,1.0,1.0,2.0,1:1,1.0,1.0,2.0,2.0
4,1.0,2021-08-14,Alavés,1–4,Real Madrid,3968.0,1,4,5,2021,...,Real Madrid,1:2,1.0,2.0,3.0,1:2,1.0,2.0,3.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,38.0,2022-05-22,Alavés,0–1,Cádiz,11138.0,0,1,1,2022,...,Cádiz,1:1,1.0,1.0,2.0,1:1,1.0,1.0,2.0,1.0
376,38.0,2022-05-22,Granada,0–0,Espanyol,17951.0,0,0,0,2022,...,Empate,1:1,1.0,1.0,2.0,1:1,1.0,1.0,2.0,2.0
377,38.0,2022-05-22,Real Sociedad,1–2,Atlético Madrid,23586.0,1,2,3,2022,...,Atlético Madrid,1:2,1.0,2.0,3.0,2:1,2.0,1.0,3.0,0.0
378,38.0,2022-05-22,Barcelona,0–2,Villarreal,54850.0,0,2,2,2022,...,Villarreal,2:2,2.0,2.0,4.0,3:1,3.0,1.0,4.0,2.0


In [29]:
CalcularError2122_V2['Error absoluto'].mean()

1.3921052631578947

### ¿Y si utilizamos una regresión? ¿Y si pensamos que el resultado de un partido depende de los puntos totales con los que llega cada equipo al enfrentamiento?

In [30]:
def puntos_local(data,temporada,fecha,equipo):
    data_usar = data[((data['Wk']<fecha)&(data['Temporada']==temporada)&((data['Local']==equipo)|(data['Visitante']==equipo)))]
    puntos = 0
    for i in range(len(data_usar)):
        ganador = str(data_usar.iloc[i]['Ganador'])
        if ganador == equipo:
            puntos += 3
        if ganador == 'Empate':
            puntos +=1
    return puntos

def puntos_visita(data,temporada,fecha,equipo):
    data_usar = data[((data['Wk']<fecha)&(data['Temporada']==temporada)&((data['Local']==equipo)|(data['Visitante']==equipo)))]
    puntos = 0
    for i in range(len(data_usar)):
        ganador = str(data_usar.iloc[i]['Ganador'])
        if ganador == equipo:
            puntos += 3
        if ganador == 'Empate':
            puntos +=1
    return puntos

In [31]:
temporadas = laliga['Temporada'].unique().tolist()
for temporada in temporadas:
    laliga['Puntos local f-1'] = laliga.apply(lambda x: puntos_local(laliga,temporada,x['Wk'],x['Local']),axis=1)
    laliga['Puntos visita f-1'] = laliga.apply(lambda x: puntos_local(laliga,temporada,x['Wk'],x['Visitante']),axis=1)

In [32]:
rm_bar2 = laliga[((laliga['Local']=='Real Madrid')&(laliga['Visitante']=='Barcelona')|
        ((laliga['Local']=='Barcelona')&(laliga['Visitante']=='Real Madrid')))].reset_index(drop=True)

In [33]:
rm_bar2

,Wk,Date,Local,Score,Visitante,Attendance,Goles Local,Goles Visitante,Total goles partido,Año,Mes,Temporada,Resultado,Ganador,Puntos local f-1,Puntos visita f-1
0,10.0,2021-10-24,Barcelona,1–2,Real Madrid,86422.0,1,2,3,2021,10,21/22,Visitante,Real Madrid,19,21
1,29.0,2022-03-20,Real Madrid,0–4,Barcelona,60017.0,0,4,4,2022,3,21/22,Visitante,Barcelona,68,63
2,7.0,2020-10-24,Barcelona,1–3,Real Madrid,NaN,1,3,4,2020,10,20/21,Visitante,Real Madrid,13,14
3,30.0,2021-04-10,Real Madrid,2–1,Barcelona,NaN,2,1,3,2021,4,20/21,Local,Real Madrid,71,66
4,10.0,2019-12-18,Barcelona,0–0,Real Madrid,93426.0,0,0,0,2019,12,19/20,Empate,Empate,19,21
5,26.0,2020-03-01,Real Madrid,2–0,Barcelona,78357.0,2,0,2,2020,3,19/20,Local,Real Madrid,59,57
6,10.0,2018-10-28,Barcelona,5–1,Real Madrid,93265.0,5,1,6,2018,10,18/19,Local,Barcelona,19,21
7,26.0,2019-03-02,Real Madrid,0–1,Barcelona,78921.0,0,1,1,2019,3,18/19,Visitante,Barcelona,59,57
8,17.0,2017-12-23,Real Madrid,0–3,Barcelona,80264.0,0,3,3,2017,12,17/18,Visitante,Barcelona,37,34
9,36.0,2018-05-06,Barcelona,2–2,Real Madrid,97939.0,2,2,4,2018,5,17/18,Empate,Empate,81,84


In [34]:
X1 = rm_bar2[['Puntos local f-1','Puntos visita f-1']]
Y1 = rm_bar2['Goles Local']

X2 = rm_bar2[['Puntos local f-1','Puntos visita f-1']]
Y2 = rm_bar2['Goles Visitante']

x_train1, x_test1, y_train1, y_test1 = train_test_split(X1, Y1, test_size=0.2)
linear_model1 = LinearRegression().fit(x_train1, y_train1)

x_train2, x_test2, y_train2, y_test2 = train_test_split(X2, Y2, test_size=0.2)
linear_model2 = LinearRegression().fit(x_train2, y_train2)

print('Training score: ', linear_model1.score(x_train1, y_train1))
print('Training score: ', linear_model2.score(x_train2, y_train2))

Training score:  0.06590891448381819
Training score:  0.03286318163401081


In [35]:
predictors1 = x_train1.columns
coef1 = pd.Series(linear_model1.coef_, predictors1).sort_values()
coef1

Puntos local f-1    -0.082873
Puntos visita f-1    0.078550
dtype: float64

In [36]:
predictors2 = x_train2.columns
coef2 = pd.Series(linear_model2.coef_, predictors2).sort_values()
coef2

Puntos visita f-1   -0.043833
Puntos local f-1     0.035443
dtype: float64

In [37]:
print(f'El intercepto del modelo 1 es {linear_model1.intercept_}')
print(f'El intercepto del modelo 2 es {linear_model2.intercept_}')

El intercepto del modelo 1 es 1.7347522716394068
El intercepto del modelo 2 es 1.7952753059356465


In [38]:
y_pred1 = linear_model1.predict(x_test1)
print(f'El coeficiente de ajuste R^2 del modelo 1 es {r2_score(y_test1, y_pred1)}')

y_pred2 = linear_model2.predict(x_test2)
print(f'El coeficiente de ajuste R^2 del modelo 2 es {r2_score(y_test2, y_pred2)}')

El coeficiente de ajuste R^2 del modelo 1 es -1.1535723359821133
El coeficiente de ajuste R^2 del modelo 2 es -0.9782067833749062


In [39]:
y_pred1

array([1.75709952, 1.80970955, 1.32258112])

In [40]:
y_pred2

array([1.40386564, 1.44393117, 1.38794186])

In [41]:
rm_bar2.sort_values(['Temporada','Wk'],ascending=False)

,Wk,Date,Local,Score,Visitante,Attendance,Goles Local,Goles Visitante,Total goles partido,Año,Mes,Temporada,Resultado,Ganador,Puntos local f-1,Puntos visita f-1
1,29.0,2022-03-20,Real Madrid,0–4,Barcelona,60017.0,0,4,4,2022,3,21/22,Visitante,Barcelona,68,63
0,10.0,2021-10-24,Barcelona,1–2,Real Madrid,86422.0,1,2,3,2021,10,21/22,Visitante,Real Madrid,19,21
3,30.0,2021-04-10,Real Madrid,2–1,Barcelona,NaN,2,1,3,2021,4,20/21,Local,Real Madrid,71,66
2,7.0,2020-10-24,Barcelona,1–3,Real Madrid,NaN,1,3,4,2020,10,20/21,Visitante,Real Madrid,13,14
5,26.0,2020-03-01,Real Madrid,2–0,Barcelona,78357.0,2,0,2,2020,3,19/20,Local,Real Madrid,59,57
4,10.0,2019-12-18,Barcelona,0–0,Real Madrid,93426.0,0,0,0,2019,12,19/20,Empate,Empate,19,21
7,26.0,2019-03-02,Real Madrid,0–1,Barcelona,78921.0,0,1,1,2019,3,18/19,Visitante,Barcelona,59,57
6,10.0,2018-10-28,Barcelona,5–1,Real Madrid,93265.0,5,1,6,2018,10,18/19,Local,Barcelona,19,21
9,36.0,2018-05-06,Barcelona,2–2,Real Madrid,97939.0,2,2,4,2018,5,17/18,Empate,Empate,81,84
8,17.0,2017-12-23,Real Madrid,0–3,Barcelona,80264.0,0,3,3,2017,12,17/18,Visitante,Barcelona,37,34


In [42]:
Y_example1_loc = linear_model1.predict([[68,63]])
Y_example1_vis = linear_model2.predict([[68,63]])
print(int(Y_example1_loc),int(Y_example1_vis))

1 1


### Pregunta 1
Si se calcula el error como el error cuadrático (diferencia entre el valor real y el pronosticado), ¿cuál de las siguientes alternativas indica el promedio del error cuadrático para el pronóstico de la cantidad de goles de la temporada 21/22?

In [43]:
temp2122 = laliga[laliga['Temporada']=='21/22']
temp2122

,Wk,Date,Local,Score,Visitante,Attendance,Goles Local,Goles Visitante,Total goles partido,Año,Mes,Temporada,Resultado,Ganador,Puntos local f-1,Puntos visita f-1
0,1.0,2021-08-13,Valencia,1–0,Getafe,9868.0,1,0,1,2021,8,21/22,Local,Valencia,0,0
1,1.0,2021-08-14,Cádiz,1–1,Levante,6941.0,1,1,2,2021,8,21/22,Empate,Empate,0,0
2,1.0,2021-08-14,Mallorca,1–1,Betis,6279.0,1,1,2,2021,8,21/22,Empate,Empate,0,0
3,1.0,2021-08-14,Osasuna,0–0,Espanyol,6715.0,0,0,0,2021,8,21/22,Empate,Empate,0,0
4,1.0,2021-08-14,Alavés,1–4,Real Madrid,3968.0,1,4,5,2021,8,21/22,Visitante,Real Madrid,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,38.0,2022-05-22,Alavés,0–1,Cádiz,11138.0,0,1,1,2022,5,21/22,Visitante,Cádiz,54,0
376,38.0,2022-05-22,Granada,0–0,Espanyol,17951.0,0,0,0,2022,5,21/22,Empate,Empate,20,53
377,38.0,2022-05-22,Real Sociedad,1–2,Atlético Madrid,23586.0,1,2,3,2022,5,21/22,Visitante,Atlético Madrid,63,75
378,38.0,2022-05-22,Barcelona,0–2,Villarreal,54850.0,0,2,2,2022,5,21/22,Visitante,Villarreal,87,64


In [44]:
dt_l

,Equipo,Goles promedio de local,Goles promedio de visitante,Goles promedio
0,Alavés,1.0,1.0,1.0
1,Athletic Club,1.0,1.0,1.0
2,Atlético Madrid,2.0,1.0,2.0
3,Barcelona,3.0,2.0,2.0
4,Betis,2.0,1.0,1.0
5,Celta Vigo,2.0,1.0,1.0
6,Cádiz,1.0,1.0,1.0
7,Eibar,1.0,1.0,1.0
8,Elche,1.0,1.0,1.0
9,Espanyol,1.0,1.0,1.0


In [45]:
temp2122 = nuevo_resultado_laliga(dt_l,temp2122)
temp2122.sort_values('Temporada',ascending=False)

,Wk,Date,Local,Score,Visitante,Attendance,Goles Local,Goles Visitante,Total goles partido,Año,Mes,Temporada,Resultado,Ganador,Puntos local f-1,Puntos visita f-1,Resultado P1,Goles Local P1,Goles Visitante P1,Total goles partido P1
0,1.0,2021-08-13,Valencia,1–0,Getafe,9868.0,1,0,1,2021,8,21/22,Local,Valencia,0,0,1:1,1.0,1.0,2.0
250,26.0,2022-02-26,Getafe,2–2,Alavés,7720.0,2,2,4,2022,2,21/22,Empate,Empate,0,33,1:1,1.0,1.0,2.0
259,27.0,2022-03-04,Alavés,0–0,Sevilla,13121.0,0,0,0,2022,3,21/22,Empate,Empate,34,56,1:2,1.0,2.0,3.0
258,21.0,2022-03-02,Mallorca,0–2,Real Sociedad,11550.0,0,2,2,2022,3,21/22,Visitante,Real Sociedad,0,35,1:1,1.0,1.0,2.0
257,26.0,2022-02-28,Granada,0–0,Cádiz,14785.0,0,0,0,2022,2,21/22,Empate,Empate,19,0,1:1,1.0,1.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124,13.0,2021-11-07,Mallorca,2–2,Elche,12933.0,2,2,4,2021,11,21/22,Empate,Empate,0,0,1:1,1.0,1.0,2.0
123,13.0,2021-11-07,Osasuna,0–2,Real Sociedad,21741.0,0,2,2,2021,11,21/22,Visitante,Real Sociedad,7,22,1:1,1.0,1.0,2.0
122,13.0,2021-11-07,Valencia,3–3,Atlético Madrid,37472.0,3,3,6,2021,11,21/22,Empate,Empate,11,21,1:2,1.0,2.0,3.0
121,13.0,2021-11-07,Villarreal,1–0,Getafe,13504.0,1,0,1,2021,11,21/22,Local,Villarreal,22,0,2:1,2.0,1.0,3.0


In [46]:
temp2122['Error cuadrático'] = (temp2122['Total goles partido']-temp2122['Total goles partido P1'])*(temp2122['Total goles partido']-temp2122['Total goles partido P1'])

In [47]:
temp2122[['Error cuadrático']]

,Error cuadrático
0,1.0
1,0.0
2,0.0
3,4.0
4,4.0
...,...
375,1.0
376,4.0
377,0.0
378,4.0


In [48]:
np.round(temp2122['Error cuadrático'].mean(),2)

3.14

### Pregunta 2
Utilice los goles totales de local y de visita convertidos por cada equipo en las temporadas anteriores a la 21/22 para pronosticar cuál será el equipo que ganará el partido o si será empate. Específicamente, el que tenga más goles totales en las temporadas anteriores será el equipo pronosticado como ganador. De ser igual la cantidad de goles, se pronosticará empate. Bajo este escenario, ¿cuál de las siguientes alternativas corresponde al porcentaje de aciertos, para la temporada 21/22, de este método de pronóstico?

In [49]:
temp2122_2 = laliga[laliga['Temporada']=='21/22']
laligaS2121 = laliga[laliga['Temporada']!='21/22']
test1_p2 = laligaS2121.groupby('Local')['Goles Local'].sum().reset_index()
test1_p2 = test1_p2.rename(columns={"Local": "Equipo", 'Goles Local': 'Total goles local'})

test2_p2 = laligaS2121.groupby('Visitante')['Goles Visitante'].sum().reset_index()
test2_p2 = test2_p2.rename(columns = {'Goles Visitante': 'Total goles visitante'})

result_p2 = pd.concat([test1_p2, test2_p2], axis=1)
dt_p2 = result_p2.drop(['Visitante'], axis=1)
dt_p2['Total goles'] = (dt_p2['Total goles local'] + dt_p2['Total goles visitante'])
data_goles = dt_p2[['Equipo','Total goles local','Total goles visitante']]

In [50]:
data_goles

,Equipo,Total goles local,Total goles visitante
0,Alavés,100,90
1,Athletic Club,131,91
2,Atlético Madrid,171,130
3,Barcelona,264,212
4,Betis,143,100
5,Celta Vigo,147,110
6,Cádiz,17,19
7,Eibar,123,91
8,Elche,18,16
9,Espanyol,91,69


In [51]:
def pred_ganador(data,goles,local,visitante):
    index_loc = goles[goles['Equipo']==local].index.values.astype(int)[0]
    goles_local = goles.loc[index_loc,'Total goles local']
    index_vis = goles[goles['Equipo']==visitante].index.values.astype(int)[0]
    goles_visita = goles.loc[index_loc,'Total goles visitante']
    if goles_local>goles_visita:
        return local
    if goles_visita>goles_local:
        return visitante
    else:
        return 'Empate'

In [52]:
temp2122_2

,Wk,Date,Local,Score,Visitante,Attendance,Goles Local,Goles Visitante,Total goles partido,Año,Mes,Temporada,Resultado,Ganador,Puntos local f-1,Puntos visita f-1
0,1.0,2021-08-13,Valencia,1–0,Getafe,9868.0,1,0,1,2021,8,21/22,Local,Valencia,0,0
1,1.0,2021-08-14,Cádiz,1–1,Levante,6941.0,1,1,2,2021,8,21/22,Empate,Empate,0,0
2,1.0,2021-08-14,Mallorca,1–1,Betis,6279.0,1,1,2,2021,8,21/22,Empate,Empate,0,0
3,1.0,2021-08-14,Osasuna,0–0,Espanyol,6715.0,0,0,0,2021,8,21/22,Empate,Empate,0,0
4,1.0,2021-08-14,Alavés,1–4,Real Madrid,3968.0,1,4,5,2021,8,21/22,Visitante,Real Madrid,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,38.0,2022-05-22,Alavés,0–1,Cádiz,11138.0,0,1,1,2022,5,21/22,Visitante,Cádiz,54,0
376,38.0,2022-05-22,Granada,0–0,Espanyol,17951.0,0,0,0,2022,5,21/22,Empate,Empate,20,53
377,38.0,2022-05-22,Real Sociedad,1–2,Atlético Madrid,23586.0,1,2,3,2022,5,21/22,Visitante,Atlético Madrid,63,75
378,38.0,2022-05-22,Barcelona,0–2,Villarreal,54850.0,0,2,2,2022,5,21/22,Visitante,Villarreal,87,64


In [53]:
def acierto(data):
    if data['Ganador']==data['Pred Ganador']:
        return 1
    else:
        return 0

In [54]:
temp2122_2['Pred Ganador'] = temp2122_2.apply(lambda x:pred_ganador(temp2122_2,data_goles,x['Local'],x['Visitante']),axis=1)
temp2122_2['Acierto'] = temp2122_2.apply(lambda x: 1 if x['Ganador'] == x['Pred Ganador'] else 0,axis = 1)

In [55]:
porcentaje_acierto = len(temp2122_2[temp2122_2['Acierto']==1])/len(temp2122_2['Acierto'])*100
print(f'El porcentaje de acierto fue de {porcentaje_acierto} %')

El porcentaje de acierto fue de 44.473684210526315 %


### Pregunta 3
Si utilizamos la regresión obtenida en el archivo “VT9” (del tutorial 9), ¿cuál de las siguientes alternativas corresponde a la estimación del resultado del penúltimo partido (el de la fecha “2021-10-24”) entre el Real Madrid y el Barcelona?

In [56]:
rm_bar = temp2122_2[((temp2122_2['Local']=='Real Madrid')&(temp2122_2['Visitante']=='Barcelona')|(temp2122_2['Visitante']=='Real Madrid')&(temp2122_2['Local']=='Barcelona'))]

In [57]:
rm_bar

,Wk,Date,Local,Score,Visitante,Attendance,Goles Local,Goles Visitante,Total goles partido,Año,Mes,Temporada,Resultado,Ganador,Puntos local f-1,Puntos visita f-1,Pred Ganador,Acierto
92,10.0,2021-10-24,Barcelona,1–2,Real Madrid,86422.0,1,2,3,2021,10,21/22,Visitante,Real Madrid,19,21,Barcelona,0
288,29.0,2022-03-20,Real Madrid,0–4,Barcelona,60017.0,0,4,4,2022,3,21/22,Visitante,Barcelona,68,63,Real Madrid,0


In [58]:
ejemplo_y_loc = linear_model1.predict([[19,21]])
ejemplo_y_vis = linear_model2.predict([[19,21]])
print(f'El resultado entre Barcelona y Real Madrid fue de {int(ejemplo_y_loc)}:{int(ejemplo_y_vis)}')

El resultado entre Barcelona y Real Madrid fue de 1:1


### Pregunta 4
Usted crea una nueva columna llamada “Potencia Local Público”, que toma valor 0 si la asistencia fue menor a 88.000 y toma valor 1 cuando es igual o mayor a esta cantidad. Si usted cree que se puede predecir el resultado de un partido utilizando una regresión multivariada con respecto a esta nueva variable y a los puntos de cada equipo antes del partido, ¿cuál de las siguientes alternativas corresponde al pronóstico realizado con esta regresión para el partido del Real Madrid vs Barcelona del “2022-03-20”? Utilice solamente la temporada 21/22 para construir la regresión y el 80% de los datos para el entrenamiento. 

In [59]:
temp2122_2['Potencia Local Público'] = temp2122_2.apply(lambda x: 0 if x['Attendance']<88000 else 1,axis=1)

In [60]:
X_loc = temp2122_2[['Puntos local f-1','Puntos visita f-1','Potencia Local Público']]
Y_loc = temp2122_2['Goles Local']

X_vis = temp2122_2[['Puntos local f-1','Puntos visita f-1','Potencia Local Público']]
Y_vis = temp2122_2['Goles Visitante']

x_train_loc, x_test_loc, y_train_loc, y_test_loc = train_test_split(X_loc, Y_loc, test_size=0.2)
linear_model_loc = LinearRegression().fit(x_train_loc, y_train_loc)

x_train_vis, x_test_vis, y_train_vis, y_test_vis = train_test_split(X_vis, Y_vis, test_size=0.2)
linear_model_vis = LinearRegression().fit(x_train_vis, y_train_vis)

print('Training score: ', linear_model_loc.score(x_train_loc, y_train_loc))
print('Training score: ', linear_model_vis.score(x_train_vis, y_train_vis))

Training score:  0.041787204281051316
Training score:  0.04008523079418835


In [61]:
rm_bar

,Wk,Date,Local,Score,Visitante,Attendance,Goles Local,Goles Visitante,Total goles partido,Año,Mes,Temporada,Resultado,Ganador,Puntos local f-1,Puntos visita f-1,Pred Ganador,Acierto
92,10.0,2021-10-24,Barcelona,1–2,Real Madrid,86422.0,1,2,3,2021,10,21/22,Visitante,Real Madrid,19,21,Barcelona,0
288,29.0,2022-03-20,Real Madrid,0–4,Barcelona,60017.0,0,4,4,2022,3,21/22,Visitante,Barcelona,68,63,Real Madrid,0


In [62]:
predictors_loc = x_train_loc.columns
coef_loc = pd.Series(linear_model_loc.coef_, predictors_loc).sort_values()

predictors_vis = x_train_vis.columns
coef_vis = pd.Series(linear_model_vis.coef_, predictors_vis).sort_values()
coef_vis

Potencia Local Público   -1.029220
Puntos local f-1         -0.007491
Puntos visita f-1         0.008410
dtype: float64

In [63]:
print(f'El intercepto del modelo 1 es {linear_model_loc.intercept_}')
print(f'El intercepto del modelo 2 es {linear_model_vis.intercept_}')

y_pred_loc = linear_model_loc.predict(x_test_loc)
print(f'El coeficiente de ajuste R^2 del modelo 1 es {r2_score(y_test_loc, y_pred_loc)}')

y_pred_vis = linear_model_vis.predict(x_test_vis)
print(f'El coeficiente de ajuste R^2 del modelo 2 es {r2_score(y_test_vis, y_pred_vis)}')

El intercepto del modelo 1 es 1.4871128821913253
El intercepto del modelo 2 es 1.020810283135724
El coeficiente de ajuste R^2 del modelo 1 es 0.04025871251153823
El coeficiente de ajuste R^2 del modelo 2 es 0.013352703565860602


El resultado de la regresión solicitada es:


In [64]:
Y_example_loc = linear_model_loc.predict([[68,63,0]])
Y_example_vis = linear_model_vis.predict([[68,63,0]])
print(int(Y_example_loc),':',int(Y_example_vis))

1 : 1


### Pregunta 5
¿Cuál de las siguientes alternativas corresponde al promedio del error absoluto de aplicar la regresión multivariada propuesta en la Pregunta 4 a todos los partidos de la temporada 21/22?

In [65]:
coef_loc

Puntos visita f-1        -0.010474
Potencia Local Público    0.000000
Puntos local f-1          0.008445
dtype: float64

In [66]:
def nuevo_marcador(local, visita, ploc,pvis,asist):
    goles_loc = np.round(1.4770243466642685-0.010723*pvis+0.009038*ploc+2.533698*asist)
    goles_vis = np.round(1.1350032056206627-0.008147*pvis+0.006953*ploc+0*asist)
    return goles_loc+goles_vis

In [67]:
temp2122_2

,Wk,Date,Local,Score,Visitante,Attendance,Goles Local,Goles Visitante,Total goles partido,Año,Mes,Temporada,Resultado,Ganador,Puntos local f-1,Puntos visita f-1,Pred Ganador,Acierto,Potencia Local Público
0,1.0,2021-08-13,Valencia,1–0,Getafe,9868.0,1,0,1,2021,8,21/22,Local,Valencia,0,0,Valencia,1,0
1,1.0,2021-08-14,Cádiz,1–1,Levante,6941.0,1,1,2,2021,8,21/22,Empate,Empate,0,0,Levante,0,0
2,1.0,2021-08-14,Mallorca,1–1,Betis,6279.0,1,1,2,2021,8,21/22,Empate,Empate,0,0,Mallorca,0,0
3,1.0,2021-08-14,Osasuna,0–0,Espanyol,6715.0,0,0,0,2021,8,21/22,Empate,Empate,0,0,Osasuna,0,0
4,1.0,2021-08-14,Alavés,1–4,Real Madrid,3968.0,1,4,5,2021,8,21/22,Visitante,Real Madrid,0,0,Alavés,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,38.0,2022-05-22,Alavés,0–1,Cádiz,11138.0,0,1,1,2022,5,21/22,Visitante,Cádiz,54,0,Alavés,0,0
376,38.0,2022-05-22,Granada,0–0,Espanyol,17951.0,0,0,0,2022,5,21/22,Empate,Empate,20,53,Granada,0,0
377,38.0,2022-05-22,Real Sociedad,1–2,Atlético Madrid,23586.0,1,2,3,2022,5,21/22,Visitante,Atlético Madrid,63,75,Real Sociedad,0,0
378,38.0,2022-05-22,Barcelona,0–2,Villarreal,54850.0,0,2,2,2022,5,21/22,Visitante,Villarreal,87,64,Barcelona,0,0


In [68]:
temp2122_2['Nuevo total goles'] = temp2122_2.apply(lambda x: nuevo_marcador(x['Local'],x['Visitante'],x['Puntos local f-1'],x['Puntos visita f-1'],x['Potencia Local Público']),axis=1)
temp2122_2['Error absoluto P5']= np.abs((temp2122_2['Total goles partido']-temp2122_2['Nuevo total goles']))

In [69]:
np.round(temp2122_2['Error absoluto P5'].mean(),2)

1.39

---
# Sumativo

# P1

In [70]:
laliga2122 = pd.read_excel('datos/QS5/LaLiga21 22.xlsx', )
laliga2021 = pd.read_excel('datos/QS5/LaLiga20 21.xlsx', )
laliga1920 = pd.read_excel('datos/QS5/LaLiga19 20.xlsx', )
laliga1819 = pd.read_excel('datos/QS5/LaLiga18 19.xlsx', )
laliga1718 = pd.read_excel('datos/QS5/LaLiga17 18.xlsx', )
laliga1617 = pd.read_excel('datos/QS5/LaLiga16 17.xlsx', )

laliga = laliga2122.dropna(how='all').append(laliga2021.dropna(how='all')
).append(laliga1920.dropna(how='all')).append(laliga1819.dropna(how='all')
).append(laliga1718.dropna(how='all')).append(laliga1617.dropna(how='all')).loc[:,
['Wk','Date','Local','Score','Visitante', 'Attendance']].reset_index(drop = True)
laliga['Attendance']= laliga['Attendance']*1000

def golesvisita(marcador):
    goles = marcador.split('–')
    return int(goles[1])
def goleslocal(marcador):
    goles = marcador.split('–')
    return int(goles[0])

def torneo(año,mes):
    if ((año==2016) & (mes<=12)|(año==2017) & (mes<8)):
        return '16/17'
    elif ((año==2017) & (mes<=12)|(año==2018) & (mes<8)):
        return '17/18'
    elif ((año==2018) & (mes<=12)|(año==2019) & (mes<8)):
        return '18/19'
    elif ((año==2019) & (mes<=12)|(año==2020) & (mes<9)):
        return '19/20'
    elif ((año==2020) & (mes<=12)|(año==2021) & (mes<8)):
        return '20/21'
    elif ((año==2021) & (mes<=12)|(año==2022) & (mes<8)):
        return '21/22'

def resultado(string):
    goles = string.split('–')
    if int(goles[0])>int(goles[1]):
        return 'Local'
    if int(goles[0])<int(goles[1]):
        return 'Visitante'
    else: 
        return 'Empate'
    
def resultado_equipo(string, local, visita):
    goles = string.split('–')
    if int(goles[0])>int(goles[1]):
        return local
    if int(goles[0])<int(goles[1]):
        return visita
    else: 
        return 'Empate'
    

laliga['Goles Local'] = laliga['Score'].apply(goleslocal)
laliga['Goles Visitante'] = laliga['Score'].apply(golesvisita)
laliga['Total goles partido']= laliga['Goles Local']+laliga['Goles Visitante']
laliga['Año'] = laliga['Date'].dt.year
laliga['Mes'] = laliga['Date'].dt.month
laliga['Temporada'] = laliga.apply(lambda x: torneo(x['Año'],x['Mes']),axis = 1)
laliga['Resultado'] = laliga['Score'].apply(resultado)
laliga['Ganador'] = laliga.apply(lambda x: resultado_equipo(x['Score'],x['Local'],x['Visitante']),axis=1)

laliga

,Wk,Date,Local,Score,Visitante,Attendance,Goles Local,Goles Visitante,Total goles partido,Año,Mes,Temporada,Resultado,Ganador
0,1.0,2021-08-13,Valencia,1–0,Getafe,9868.0,1,0,1,2021,8,21/22,Local,Valencia
1,1.0,2021-08-14,Cádiz,1–1,Levante,6941.0,1,1,2,2021,8,21/22,Empate,Empate
2,1.0,2021-08-14,Mallorca,1–1,Betis,6279.0,1,1,2,2021,8,21/22,Empate,Empate
3,1.0,2021-08-14,Osasuna,0–0,Espanyol,6715.0,0,0,0,2021,8,21/22,Empate,Empate
4,1.0,2021-08-14,Alavés,1–4,Real Madrid,3968.0,1,4,5,2021,8,21/22,Visitante,Real Madrid
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2275,38.0,2017-05-21,Valencia,1–3,Villarreal,33587.0,1,3,4,2017,5,16/17,Visitante,Villarreal
2276,38.0,2017-05-21,Atlético Madrid,3–1,Athletic Club,52314.0,3,1,4,2017,5,16/17,Local,Atlético Madrid
2277,38.0,2017-05-21,Celta Vigo,2–2,Real Sociedad,14188.0,2,2,4,2017,5,16/17,Empate,Empate
2278,38.0,2017-05-21,Barcelona,4–2,Eibar,74932.0,4,2,6,2017,5,16/17,Local,Barcelona


In [71]:
equipos = laliga.Local.unique().tolist()
len(equipos)

29

In [72]:
equipos_puntos = list()
for equip in equipos:
    df_local = laliga[(laliga['Local'] == equip)]
    df_local_vict = df_local[df_local['Ganador'] == equip]
    df_visitante = laliga[(laliga['Visitante'] == equip)]
    df_visitante_vict = df_visitante[df_visitante['Ganador'] == equip]

    df_local_empt = df_local[(df_local['Ganador'] == 'Empate')]
    df_visita_empt = df_visitante[(df_visitante['Ganador'] == 'Empate')]

    pts_equip = (len(df_local_vict) * 3 ) + (len(df_visitante_vict) * 3 ) + (len(df_local_empt) * 1) + (len(df_visita_empt) * 1)
    equipos_puntos.append((equip, pts_equip))

equipos_puntos



[('Valencia', 324),
 ('Cádiz', 83),
 ('Mallorca', 72),
 ('Osasuna', 165),
 ('Alavés', 260),
 ('Celta Vigo', 271),
 ('Barcelona', 504),
 ('Sevilla', 406),
 ('Villarreal', 349),
 ('Elche', 78),
 ('Betis', 316),
 ('Granada', 160),
 ('Espanyol', 225),
 ('Athletic Club', 311),
 ('Real Sociedad', 343),
 ('Atlético Madrid', 460),
 ('Levante', 215),
 ('Getafe', 245),
 ('Rayo Vallecano', 74),
 ('Real Madrid', 494),
 ('Eibar', 224),
 ('Valladolid', 114),
 ('Huesca', 67),
 ('Leganés', 159),
 ('Girona', 88),
 ('La Coruña', 65),
 ('Málaga', 66),
 ('Las Palmas', 61),
 ('Sporting Gijón', 31)]

In [73]:
equipos_asistencia = list()

for equip in equipos:
    equipos_asist = laliga[laliga['Local'] == equip]
    asist = equipos_asist['Attendance'].sum()

    equipos_asistencia.append((equip, asist))

equipos_asistencia
    

[('Valencia', 3213405.0),
 ('Cádiz', 267049.0),
 ('Mallorca', 448182.0),
 ('Osasuna', 823258.0),
 ('Alavés', 1470820.0),
 ('Celta Vigo', 1385598.0),
 ('Barcelona', 6272438.0),
 ('Sevilla', 2974514.0),
 ('Villarreal', 1467729.0),
 ('Elche', 300678.0),
 ('Betis', 3847810.0),
 ('Granada', 755712.0),
 ('Espanyol', 1735163.0),
 ('Athletic Club', 3417075.0),
 ('Real Sociedad', 2120247.0),
 ('Atlético Madrid', 4834556.0),
 ('Levante', 1271225.0),
 ('Getafe', 737450.0),
 ('Rayo Vallecano', 958506.0),
 ('Real Madrid', 5452011.0),
 ('Eibar', 374501.0),
 ('Valladolid', 618867.0),
 ('Huesca', 125488.0),
 ('Leganés', 724010.0),
 ('Girona', 406070.0),
 ('La Coruña', 818198.0),
 ('Málaga', 813851.0),
 ('Las Palmas', 696172.0),
 ('Sporting Gijón', 438302.0)]

In [74]:
list_df = list()
for i in range(len(equipos_asistencia)):
    eq = equipos_asistencia[i][0]
    pts = equipos_puntos[i][1]
    ass = equipos_asistencia[i][1]

    list_df.append((eq, pts, ass))

df = pd.DataFrame(list_df)
df.rename(columns={0: 'Equipo', 1: 'Ptos totales', 2: 'Asistencia'}, inplace=True)
df

,Equipo,Ptos totales,Asistencia
0,Valencia,324,3213405.0
1,Cádiz,83,267049.0
2,Mallorca,72,448182.0
3,Osasuna,165,823258.0
4,Alavés,260,1470820.0
5,Celta Vigo,271,1385598.0
6,Barcelona,504,6272438.0
7,Sevilla,406,2974514.0
8,Villarreal,349,1467729.0
9,Elche,78,300678.0


In [75]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.cluster import KMeans
from sklearn import preprocessing 
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [76]:
min_max_scaler1 = preprocessing.MinMaxScaler() 


df_1 = df[['Ptos totales', 'Asistencia']]
df_1
df_escalado = min_max_scaler1.fit_transform(df_1)
df_escalado = pd.DataFrame(df_escalado)
df_escalado = df_escalado.rename(columns = {0: 'Ptos totales', 1:'Asistencia'}) 
df_escalado

,Ptos totales,Asistencia
0,0.619450,0.502349
1,0.109937,0.023029
2,0.086681,0.052497
3,0.283298,0.113515
4,0.484144,0.218862
5,0.507400,0.204998
6,1.000000,1.000000
7,0.792812,0.463486
8,0.672304,0.218359
9,0.099366,0.028500


In [77]:
x1 = df_escalado['Ptos totales'].values
y1 = df_escalado['Asistencia'].values

In [78]:
kmeans = KMeans(n_clusters=4).fit(df_escalado)
centroids = kmeans.cluster_centers_
centroids

array([[0.08802614, 0.06671446],
       [0.96194503, 0.87753769],
       [0.6564482 , 0.44162422],
       [0.38642236, 0.14728795]])

In [79]:
labels = kmeans.predict(df_escalado)
df['label'] = labels
df

,Equipo,Ptos totales,Asistencia,label
0,Valencia,324,3213405.0,2
1,Cádiz,83,267049.0,0
2,Mallorca,72,448182.0,0
3,Osasuna,165,823258.0,3
4,Alavés,260,1470820.0,3
5,Celta Vigo,271,1385598.0,3
6,Barcelona,504,6272438.0,1
7,Sevilla,406,2974514.0,2
8,Villarreal,349,1467729.0,2
9,Elche,78,300678.0,0


In [80]:
df[df['Equipo'] == 'Sevilla']

,Equipo,Ptos totales,Asistencia,label
7,Sevilla,406,2974514.0,2


In [81]:
df.groupby('label').count()

,Equipo,Ptos totales,Asistencia
label,,,
0,11,11,11
1,3,3,3
2,6,6,6
3,9,9,9


In [82]:
laliga_2122 = laliga[laliga['Temporada'] == '21/22']
laliga_2122

,Wk,Date,Local,Score,Visitante,Attendance,Goles Local,Goles Visitante,Total goles partido,Año,Mes,Temporada,Resultado,Ganador
0,1.0,2021-08-13,Valencia,1–0,Getafe,9868.0,1,0,1,2021,8,21/22,Local,Valencia
1,1.0,2021-08-14,Cádiz,1–1,Levante,6941.0,1,1,2,2021,8,21/22,Empate,Empate
2,1.0,2021-08-14,Mallorca,1–1,Betis,6279.0,1,1,2,2021,8,21/22,Empate,Empate
3,1.0,2021-08-14,Osasuna,0–0,Espanyol,6715.0,0,0,0,2021,8,21/22,Empate,Empate
4,1.0,2021-08-14,Alavés,1–4,Real Madrid,3968.0,1,4,5,2021,8,21/22,Visitante,Real Madrid
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,38.0,2022-05-22,Alavés,0–1,Cádiz,11138.0,0,1,1,2022,5,21/22,Visitante,Cádiz
376,38.0,2022-05-22,Granada,0–0,Espanyol,17951.0,0,0,0,2022,5,21/22,Empate,Empate
377,38.0,2022-05-22,Real Sociedad,1–2,Atlético Madrid,23586.0,1,2,3,2022,5,21/22,Visitante,Atlético Madrid
378,38.0,2022-05-22,Barcelona,0–2,Villarreal,54850.0,0,2,2,2022,5,21/22,Visitante,Villarreal


In [83]:
def get_goals(row, df):
    team_1 = row['Local']
    team_2 = row['Visitante']
    if row['Wk'] == 1.0:
        return row['Goles Local'] + row['Goles Visitante']
    
    wk_before = row['Wk'] - 1.0
    team_1_df_filter = df[(df['Wk'] == wk_before) & ((df['Local'] == team_1) | (df['Visitante'] == team_1))]
    print(team_1)
    print(team_2)

laliga_2122['Total goles'] = laliga_2122['Goles Local'] + laliga_2122['Goles Visitante']
laliga_2122['Pred goles'] = laliga.apply(lambda x: get_goals(x, laliga_2122), axis=1)

Valencia
Getafe
Cádiz
Levante
Mallorca
Betis
Osasuna
Espanyol
Alavés
Real Madrid
Celta Vigo
Atlético Madrid
Barcelona
Real Sociedad
Sevilla
Rayo Vallecano
Villarreal
Granada
Elche
Athletic Club
Betis
Cádiz
Alavés
Mallorca
Granada
Valencia
Espanyol
Villarreal
Athletic Club
Barcelona
Real Sociedad
Rayo Vallecano
Atlético Madrid
Elche
Levante
Real Madrid
Getafe
Sevilla
Osasuna
Celta Vigo
Mallorca
Espanyol
Valencia
Alavés
Celta Vigo
Athletic Club
Real Sociedad
Levante
Elche
Sevilla
Betis
Real Madrid
Barcelona
Getafe
Rayo Vallecano
Granada
Cádiz
Osasuna
Atlético Madrid
Villarreal
Levante
Rayo Vallecano
Athletic Club
Mallorca
Espanyol
Atlético Madrid
Osasuna
Valencia
Cádiz
Real Sociedad
Real Madrid
Celta Vigo
Getafe
Elche
Granada
Betis
Celta Vigo
Cádiz
Rayo Vallecano
Getafe
Atlético Madrid
Athletic Club
Elche
Levante
Alavés
Osasuna
Mallorca
Villarreal
Real Sociedad
Sevilla
Betis
Espanyol
Valencia
Real Madrid
Barcelona
Granada
Getafe
Atlético Madrid
Levante
Celta Vigo
Athletic Club
Rayo Valle